# A first look at data

date: 12/04/2018

Example code for reading and checking data


In [ ]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde
import os

filepath='/home/ioana/src/icecube/Noemi/stage/cic_att' 
os.chdir(filepath)
%run data_functions.py
from data_functions import obtain_attenuation

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (10.0, 10.0)    # resize plots

This is the part that reads data and makes a basic selection

In [ ]:
df = pd.DataFrame()
df = pd.read_hdf('/data/IceCube/merged2015.h5', key='data')
df['cos2'] = np.cos(df.zenith)**2
df['I'] = 0
df['s125_error'] = 0.1*df['s125']

df.drop(df.columns.difference(['s125', 's125_error',"cos2", "I","zenith"]), 1, inplace= True)
df= df.loc[df.s125>25]
df= df.loc[df.cos2>0.5]  
df.reset_index(inplace=True)
print('You have selected {} events'.format(df.s125.count()))



In [ ]:
n_bins = 10
intensity = 200
samples = 100

fit_results, fitted_data = obtain_attenuation(df, n_bins, intensity, samples )

In [ ]:
import math
groups = fitted_data
sample = fit_results
cos_ref = np.cos(math.radians(25))**2
cos2 = np.linspace(0.5, 1, 20)-cos_ref

a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                  zip(*np.percentile(sample, [16, 50, 84],
                                      axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s_ref = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

# Plot a subset of the samples
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")
for a, b, f in sample[np.random.randint(len(sample), size=250)]:
    plt.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="b", alpha=0.03)
    plt.plot(cos2, s38_mcmc[0] * (b_mcmc[0] * cos2**2 + a_mcmc[0] * cos2 + 1), color="b", lw=0.5, alpha=0.8)
plt.errorbar(groups.cos2.mean()-cos_ref, groups.s125.mean(), yerr= groups.s125.std().tolist(), fmt=".k")

In [ ]:
import corner
fig = corner.corner(sample, labels=["$a$","$b$", "$s_{ref}$"], quantiles=[0.16, 0.84],
                    show_titles=True, title_kwargs={"fontsize": 20}, color='b')
#fig.savefig("")
a_mcmc,  b_mcmc,  s38_mcmc  = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(sample, [16, 50, 84],
                                                axis=0)))

#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s38 = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))

In [ ]:
fig2, ax2 =plt.subplots(figsize=(10,6))
temp, groups = set_intensity(df, 4)
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")

plt.legend()


  

In [ ]:
%run data_functions.py
from data_functions import set_intensity
df['s38'] = df.s125/(b_mcmc[0] * (df.cos2-cos_ref)**2 + a_mcmc[0] * (df.cos2-cos_ref) + 1)

print(df)
temp, groups = set_intensity(df, 4, 's38')
for name, group in groups:
    ax2.plot(np.log10(group.s125), group.I, 
             label='$theta$ = {}'.format(math.floor(math.degrees(group.zenith.mean()))))
ax2.set_yscale("log", nonposy='clip')
plt.xlabel("$log_{10}(S_{125}/VEM)$")
plt.ylabel("Intensity  [a.u.]")

plt.legend()